In [128]:
import pandas as pd
import matplotlib.pyplot as plt 
import re
import textdistance
import numpy as np
%matplotlib inline

In [125]:
# !pip install textdistance

Looking in indexes: http://as-pypi-gcs


In [2]:
ee = pd.read_csv('../../data/processed/elevator_to_line_dir_station.csv')

In [41]:
ee[ee.station.str.contains('34 St - Penn')]

,Unnamed: 0,equipment_id,line,direction,station
105,40,EL214,1,north,34 St - Penn
106,247,EL215,3,south,34 St - Penn
107,665,EL215,2,south,34 St - Penn
108,456,EL215,2,north,34 St - Penn
109,38,EL215,3,north,34 St - Penn
110,39,EL216,1,south,34 St - Penn
129,37,EL225,E,north,34 St - Penn
130,246,EL225,C,north,34 St - Penn
131,252,EL226,C,north,34 St - Penn
132,43,EL226,E,north,34 St - Penn


In [350]:
ee.direction.unique()

array(['south', 'north', 'manhattan', 'wakefield 241 st', 'woodlawn',
       'norwood 205 st', 'jamaica', 'bay ridge',
       'flatbush av brooklyn college', '637N, 637S',
       'brighton beach, coney island, bay ridge', 'coney island',
       'coney island, brighton beach', 'jamaica center',
       'flushing main st', 'flushing', 'jamaica, forest hills',
       'manhattan-bound', 'far rockaway', 'brighton beach/coney island'],
      dtype=object)

In [4]:
stops = pd.read_csv('../../data/external/station_connections/nyctsubwaystops_100627_curcleaned.csv')

In [5]:
stops.head()

,the_geom,zone_id,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,cartodb_id,stop_url,location_t,parent_sta,name_cur,routes_wkd,routes_off,routes_all
0,0101000020E6100000C9022670EB7B52C0CEF9298E0369...,0,302,0,145TH STREET-LENOX,0,40.820421,-73.936245,94,0,0,0,145TH STREET,3,3,3
1,0101000020E6100000FBCE2F4AD07D52C08DC877297557...,0,A44,0,CLINTON - WASHINGTON,0,40.683263,-73.965838,212,0,0,0,CLINTON - WASHINGTON,C,"A, C","A, C"
2,0101000020E610000010AE80423D7F52C0DEC5FB71FB5B...,0,F15,0,DELANCY STREET,0,40.718611,-73.988114,299,0,0,0,DELANCY STREET,F,F,F
3,0101000020E6100000766F4562827952C04704E3E0D271...,0,101,0,242ND STREET-BWAY (Van Cortland Park),0,40.889248,-73.898583,1,0,0,0,242ND STREET - VAN CORTLAND PARK,1,1,1
4,0101000020E6100000E6EB32FCA77952C076A73B4F3C71...,0,103,0,238TH STREET-BWAY,0,40.884653,-73.900878,2,0,0,0,238TH STREET,1,1,1


In [23]:
stops.dtypes

the_geom       object
zone_id         int64
stop_id        object
stop_code       int64
stop_name      object
stop_desc       int64
stop_lat      float64
stop_lon      float64
cartodb_id      int64
stop_url        int64
location_t      int64
parent_sta      int64
name_cur       object
routes_wkd     object
routes_off     object
routes_all     object
dtype: object

In [28]:
stops[stops.stop_id.str.rfind('S',0,1) != -1]

,the_geom,zone_id,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,cartodb_id,stop_url,location_t,parent_sta,name_cur,routes_wkd,routes_off,routes_all
61,0101000020E6100000F6CFD380418F52C0F9BCE2A94742...,0,S12,0,NASSAU,0,40.517812,-74.238373,491,0,0,0,NASSAU,SI,SI,SI
469,0101000020E6100000906802452C7D52C0EBE40CC51D57...,0,S01,0,FRANKLIN - FULTON,0,40.680596,-73.955827,469,0,0,0,FRANKLIN - FULTON,FS,FS,FS
470,0101000020E610000057772CB6497D52C0D420CCED5E56...,0,S03,0,PARK PLACE,0,40.674772,-73.957624,470,0,0,0,PARK PLACE,FS,FS,FS
471,0101000020E61000007FF62345647D52C0200DA7CCCD55...,0,S04,0,BOTANIC GARDENS - EASTERN PKWY,0,40.670343,-73.959245,471,0,0,0,BOTANIC GARDENS - EASTERN PKWY,FS,FS,FS
472,0101000020E610000014252191B68452C0C56E9F556652...,0,S31,0,SAINT GEORGE TERMINAL,0,40.643748,-74.073643,472,0,0,0,SAINT GEORGE TERMINAL,SI,SI,SI
473,0101000020E61000002766BD18CA8452C0251C7A8B8751...,0,S30,0,TOMPKINSVILLE,0,40.636949,-74.074835,473,0,0,0,TOMPKINSVILLE,SI,SI,SI
474,0101000020E6100000BAF94674CF8452C07CD5CA845F50...,0,S29,0,STAPLETON,0,40.627915,-74.075162,474,0,0,0,STAPLETON,SI,SI,SI
475,0101000020E61000009EB7B1D9918452C014B18861874F...,0,S28,0,CLIFTON,0,40.621319,-74.071402,475,0,0,0,CLIFTON,SI,SI,SI
476,0101000020E610000033C170AE618552C0AF5417F0324D...,0,S27,0,GRASMERE,0,40.603117,-74.084087,476,0,0,0,GRASMERE,SI,SI,SI
477,0101000020E6100000E2ADF36F978552C0583332C85D4C...,0,S26,0,OLD TOWN,0,40.596612,-74.087368,477,0,0,0,OLD TOWN,SI,SI,SI


In [19]:
stops[stops.stop_name.str.contains('34TH')].head()

,the_geom,zone_id,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,cartodb_id,stop_url,location_t,parent_sta,name_cur,routes_wkd,routes_off,routes_all
28,0101000020E610000033DE567A6D7F52C09666F3380C60...,0,128,0,34TH ST. - PENN STA.,0,40.750373,-73.991057,26,0,0,0,34TH ST - PENN STA,"1, 2, 3","1, 2, 3","1, 2, 3"
203,0101000020E61000000249D8B7937F52C05D1ABFF04A60...,0,A28,0,34TH ST - PENN STATION,0,40.752287,-73.993391,200,0,0,0,34TH ST - PENN STATION,"A, C, E","A, C, E","A, C, E"
263,0101000020E610000025CFF57D387F52C04C18CDCAF65F...,0,D17,0,34TH STREET - HERALD SQ,0,40.749719,-73.987823,262,0,0,0,34TH STREET - HERALD SQ,"B, D, F, M","B, D, F, M","B, D, F, M"
442,0101000020E61000005305A3923A7F52C09694BBCFF15F...,0,R17,0,34TH STREET - HERALD SQ,0,40.749567,-73.987950,442,0,0,0,34TH STREET - HERALD SQ,"N, Q, R","N, Q, R","N, Q, R"


In [212]:
routes = pd.read()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,1,MTA NYCT,1,Broadway - 7 Avenue Local,Trains operate between 242 St in the Bronx and...,1,http://web.mta.info/nyct/service/pdf/t1cur.pdf,EE352E,NaN
1,2,MTA NYCT,2,7 Avenue Express,"Trains operate between Wakefield-241 St, Bronx...",1,http://web.mta.info/nyct/service/pdf/t2cur.pdf,EE352E,
2,3,MTA NYCT,3,7 Avenue Express,"Trains operate between 148 St, 7 Av, Manhattan...",1,http://web.mta.info/nyct/service/pdf/t3cur.pdf,EE352E,NaN
3,4,MTA NYCT,4,Lexington Avenue Express,Trains operate daily between Woodlawn/Jerome A...,1,http://web.mta.info/nyct/service/pdf/t4cur.pdf,00933C,NaN
4,5,MTA NYCT,5,Lexington Avenue Express,"Weekdays daytime, most trains operate between ...",1,http://web.mta.info/nyct/service/pdf/t5cur.pdf,00933C,NaN
5,5X,MTA NYCT,5X,Lexington Avenue Express,"Weekdays daytime, most trains operate between ...",1,http://web.mta.info/nyct/service/pdf/t5cur.pdf,00933C,NaN
6,6,MTA NYCT,6,Lexington Avenue Local,Local trains operate between Pelham Bay Park/B...,1,http://web.mta.info/nyct/service/pdf/t6cur.pdf,00933C,NaN
7,6X,MTA NYCT,6X,Pelham Bay Park Express,Express trains operate between Pelham Bay Park...,1,http://web.mta.info/nyct/service/pdf/t6cur.pdf,00A65C,NaN
8,7,MTA NYCT,7,Flushing Local,"Trains operate between Main St-Flushing, Queen...",1,http://web.mta.info/nyct/service/pdf/t7cur.pdf,B933AD,NaN
9,7X,MTA NYCT,7X,Flushing Express,"Trains operate between Main St-Flushing, Queen...",1,http://web.mta.info/nyct/service/pdf/t7cur.pdf,B933AD,NaN


In [214]:
trips[trips.route_id == 'W']

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
16420,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_037300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16421,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_039300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16422,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_040300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16423,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_041300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16424,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_042300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16425,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_042350_N..N70R,Astoria - Ditmars Blvd,0,NaN,NaN
16426,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_042600_N..N72R,Astoria - Ditmars Blvd,0,NaN,NaN
16427,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_043300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16428,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_044300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16429,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_044350_N..N72R,Astoria - Ditmars Blvd,0,NaN,NaN


In [ ]:
'''
Subset routes_wkd to contain line, fuzzy match station name (to upper) with stop_name and name_cur
'''

In [228]:
stops_new = pd.read_csv('../../data/raw/google_transit/stops.txt')

In [229]:
stops_new.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
3,103,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,1,NaN
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103


In [230]:
stops_new[stops_new.stop_name == '30 Av']

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1314,R04,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,1,NaN
1315,R04N,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,0,R04
1316,R04S,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,0,R04


In [231]:
platforms = stops_new[stops_new.location_type == 0]

In [232]:
platforms.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
5,103S,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
7,104N,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104


In [233]:
platforms[platforms.stop_name.str.contains('34 St - Penn Station')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
76,128N,NaN,34 St - Penn Station,NaN,40.750373,-73.991057,NaN,NaN,0,128
77,128S,NaN,34 St - Penn Station,NaN,40.750373,-73.991057,NaN,NaN,0,128
601,A28N,NaN,34 St - Penn Station,NaN,40.752287,-73.993391,NaN,NaN,0,A28
602,A28S,NaN,34 St - Penn Station,NaN,40.752287,-73.993391,NaN,NaN,0,A28


In [234]:
stop_times = pd.read_csv('../../data/raw/google_transit/stop_times.txt')

In [352]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,NaN,0,0,NaN
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,NaN,0,0,NaN
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,NaN,0,0,NaN
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,NaN,0,0,NaN
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,NaN,0,0,NaN
5,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:13:00,00:13:00,108S,6,NaN,0,0,NaN
6,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:14:30,00:14:30,109S,7,NaN,0,0,NaN
7,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:16:00,00:16:00,110S,8,NaN,0,0,NaN
8,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:17:30,00:17:30,111S,9,NaN,0,0,NaN
9,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:19:30,00:19:30,112S,10,NaN,1,1,NaN


In [65]:
stop_times['line'] = [x[x.find('..')-1:x.find('..')] for x in stop_times.trip_id]

In [238]:
weekday = stop_times[stop_times.trip_id.str.contains('Weekday')]

In [248]:
routes_subset = routes[(routes.route_id.str.len() == 1) & (routes.route_id != 'H')]

In [239]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_000600_1..S03R,South Ferry,1,NaN,1..S03R
1,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_002600_1..S03R,South Ferry,1,NaN,1..S03R
2,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_004600_1..S03R,South Ferry,1,NaN,1..S03R
3,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_006600_1..S03R,South Ferry,1,NaN,1..S03R
4,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_007200_1..N03R,Van Cortlandt Park - 242 St,0,NaN,1..N03R


In [249]:
weekday = weekday.merge(trips[trips.route_id.isin(routes_subset.route_id)][['trip_id','route_id']],on="trip_id")

In [251]:
unique_stop_ids = pd.DataFrame(weekday.stop_id.unique(),columns=['stop_id'])

In [252]:
unique_stop_ids['routes_wkd'] = [''.join(weekday[weekday.stop_id == x]['route_id'].unique()) for x in unique_stop_ids.stop_id]

In [253]:
platforms = platforms.merge(unique_stop_ids,on='stop_id')

In [254]:
platforms.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,routes_wkd
0,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,1
1,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,1
2,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103,1
3,103S,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103,1
4,104N,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104,1


In [268]:
ee = pd.read_csv('../../data/processed/elevator_to_line_dir_station.csv')

In [351]:
ee.head()

,Unnamed: 0,Unnamed: 0.1,equipment_id,line,direction,station,possible_stops,stop_id
0,0,230,EL103,1,south,191 St,"110N, 110S",110S
1,1,21,EL103,1,north,191 St,"110N, 110S",110N
2,2,185,EL122,6,manhattan,Pelham Bay Park,"601N, 601S",NaN
3,3,18,EL124,A,north,175 St,"A07N, A07S",A07N
4,4,628,EL126,5,south,125 St,"621N, 621S",621S


In [269]:
ee['stop_ids'] = ''

In [270]:
for index, row in ee.iterrows():
    if row.stop_ids == '':
        subset = platforms[platforms.routes_wkd.str.contains(row.line)]
        if subset.shape[0] > 0:
            subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
            name_dist = [max(textdistance.jaro_winkler(row.station,y),textdistance.jaccard(row.station,y)) for y in subset_stop_names.stop_name]
                
            matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
            matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
            score = max(name_dist)
            if score > 0.79:
                ee.loc[index,'possible_stops'] = ', '.join(matched_stop_ids.stop_id)
                print('{} : {} - {}'.format(row.station,matched_station_name,score))


191 St : 191 St - 1
191 St : 191 St - 1
Pelham Bay Park : Pelham Bay Park - 1
175 St : 175 St - 1
125 St : 125 St - 1
125 St : 125 St - 1
125 St : 125 St - 1
125 St : 125 St - 1
125 St : 125 St - 1
125 St : 125 St - 1
Simpson St : Simpson St - 1
Simpson St : Simpson St - 1
Simpson St : Simpson St - 1
Simpson St : Simpson St - 1
149 St - 3 Av : 3 Av - 149 St - 1.0
149 St - 3 Av : 3 Av - 149 St - 1.0
149 St - 3 Av : 3 Av - 149 St - 1.0
149 St - 3 Av : 3 Av - 149 St - 1.0
161 St - Yankee Stadium : 161 St - Yankee Stadium - 1
161 St - Yankee Stadium : 161 St - Yankee Stadium - 1
161 St - Yankee Stadium : 161 St - Yankee Stadium - 1
161 St - Yankee Stadium : 161 St - Yankee Stadium - 1
161 St - Yankee Stadium : 161 St - Yankee Stadium - 1
161 St - Yankee Stadium : 161 St - Yankee Stadium - 1
Pelham Pkwy : Pelham Pkwy - 1
Pelham Pkwy : Pelham Pkwy - 1
Pelham Pkwy : Pelham Pkwy - 1
Pelham Pkwy : Pelham Pkwy - 1
168 St : 168 St - 1
168 St : 168 St - 1
168 St : 168 St - 1
168 St : 168 St - 1
12

In [278]:
ee[(ee.stop_ids == '') & (ee.station == 'Broadway-Lafayette/Bleecker St')] = '637N, 637S'

In [267]:
# for index, row in ee.iterrows():
#     if row.stop_ids == '':
#         subset = platforms[platforms.routes_wkd.str.contains(row.line)]
#         if subset.shape[0] > 0:
#             subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
#             name_dist = [textdistance.jaro_winkler(row.station,y) for y in subset_stop_names.stop_name]
#             matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
#             matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
#             score = max(name_dist)
#             if score > 0.5:
# #                 ee.iloc[index,5] = ', '.join(matched_stop_ids.stop_id)
#                 print('{} : {} - {}'.format(row.station,matched_station_name,score))

Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq 

In [279]:
ee[ee.stop_ids == ''].shape[0]/ee.shape[0]

0.025134649910233394

In [282]:
ee.head()

,Unnamed: 0,equipment_id,line,direction,station,stop_ids
0,230,EL103,1,south,191 St,"110N, 110S"
1,21,EL103,1,north,191 St,"110N, 110S"
2,185,EL122,6,manhattan,Pelham Bay Park,"601N, 601S"
3,18,EL124,A,north,175 St,"A07N, A07S"
4,628,EL126,5,south,125 St,"621N, 621S"


In [301]:
pattern = re.compile('.*N')
[pattern.match(x) for x in ee.stop_ids[0].split(',')]
# [re.findall('N',x) for x in ee.stop_ids[0].split(',')]

[<_sre.SRE_Match object; span=(0, 4), match='110N'>, None]

In [305]:
[x for x in ee.stop_ids[0].split(', ') if re.match('.*N$',x)]

['110N']

'S'

In [338]:
def get_dir(df):
#     print(df)
    matches = [x for x in df.stop_ids.split(', ') if re.match('.*{}$'.format(df.direction[0].upper()),x)]
    if matches:
        return matches[0]
    else: 
        return 'NULL'

In [340]:
ee['stop_id'] = ee.apply(get_dir,axis=1)

In [343]:
ee.rename(columns={'stop_ids':'possible_stops'},inplace=True)

In [344]:
ee.to_csv('../../data/processed/elevator_to_GTFS_KM.csv')

In [353]:
ee_w_stops = pd.read_csv('../../data/processed/elevator_to_GTFS_KM.csv')

In [355]:
ee_w_stops.shape

(557, 8)

In [357]:
ee_w_stops[ee_w_stops.stop_id.isnull()]

,Unnamed: 0,Unnamed: 0.1,equipment_id,line,direction,station,possible_stops,stop_id
2,2,185,EL122,6,manhattan,Pelham Bay Park,"601N, 601S",NaN
7,7,1464,EL126,METRO-NORTH,south,125 St,NaN,NaN
8,8,1255,EL126,METRO-NORTH,north,125 St,NaN,NaN
12,12,404,EL127,5,wakefield 241 st,Simpson St,"217N, 217S",NaN
13,13,195,EL127,2,wakefield 241 st,Simpson St,"217N, 217S",NaN
14,14,403,EL128,5,manhattan,Simpson St,"217N, 217S",NaN
15,15,194,EL128,2,manhattan,Simpson St,"217N, 217S",NaN
18,18,220,EL130,5,manhattan,149 St - 3 Av,"221N, 221S",NaN
19,19,11,EL130,2,manhattan,149 St - 3 Av,"221N, 221S",NaN
20,20,15,EL132,4,woodlawn,161 St - Yankee Stadium,"414N, 414S",NaN
